In [13]:
import pandas as pd
import sys; sys.path.insert(0, '..')
from src.arcs import generate_arc
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import re
import numpy as np

In [14]:
data = pd.read_csv("../data/char_details.csv")

In [15]:
data.head()

,Name,affiliation,age,bounty,epithet,first,occupation,status,residence,dfname
0,Shanks,Red Hair Pirates;[2] Four Emperors;[3] Roger P...,27 (Chapter 1); 37 (Chapter 1 to 597);[6] 39 (...,"4,048,900,000[10]","""Red-Haired Shanks"" (赤髪のシャンクス, Akagami no Shan...",Chapter 1; Episode 4[1],Pirate Chief;[5] Apprentice (former)[4],Alive,NaN,NaN
1,Benn Beckman,Red Hair Pirates[1],38 (Chapter 1);48 (Chapter 1 to 597);50 (Chapt...,Unknown[5],NaN,Chapter 1; Episode 4[1],Pirate;[1] First Mate[2],Alive,NaN,NaN
2,Lucky Roux,Red Hair Pirates[1],23 (Chapter 1)33 (Chapter 1 to 597)35 (Chapter...,Unknown[2],NaN,Chapter 1; Episode 4[1],Pirate Officer[1]; Combatant[2],Alive,NaN,NaN
3,Yasopp,Red Hair Pirates[1],35 (Chapter 1);45 (Chapter 96 to 597)47 (Chapt...,Unknown[5],"""Chaser"" (追擊者（チェイサー）, Cheisā?)[3]",Chapter 1; Episode 4[1],Pirate Officer; Sniper[2],Alive,NaN,NaN
4,Gol D. Roger,Roger Pirates[1],NaN,"5,564,800,000[5]","Gold Roger (ゴールド・ロジャー, Gōrudo Rojā?),[1]Pirate...",Chapter 1; We Are![1],Pirate;[1] Captain,Deceased,Loguetown (former),NaN


In [16]:
data['status'].value_counts()

Alive        939
Deceased      95
Unknown       71
Active         4
Destroyed      1
Name: status, dtype: int64

In [17]:
def get_last_known_bounty(row):
    if type(row) == float:
        return row
    elif type(row) == str:
        x = re.sub(r"\[.*?\]", " ", row)
        x = x.split(" ")
        ret = ''.join([n for n in x[0] if n.isdigit()])
        if len(ret) ==0:
            return np.nan
        return int(ret)

data['last_bounty'] = data['bounty'].apply(get_last_known_bounty)

In [18]:
fig = px.histogram(data, x="last_bounty", nbins = 100)

In [19]:
fig.show()

In [20]:
def get_latest_age(row):
    if type(row) == str:
        x = re.sub(r"\[.*?\]", " ", row)
        x = re.sub(r"\(.*?\)", " ", x)
        x = x.replace(";", "")
        x = x.split(" ")
        
        ret = ' '.join([n for n in x if n.isdigit()])
        ret = ret.split(" ")
        newret = []
        for i in ret:
            try:
                newret.append(int(i))
            except:
                newret.append(i)

        return (max(newret))

def get_main_crew(row):
    if type(row) == str:
        x = re.sub(r"\[.*?\]", " ", row)
        x = re.sub(r"\(.*?\)", " ", x)
        x = x.split(";")
        # x = x.split("")
        return x[0]

data['main_crew'] = data['affiliation'].apply(get_main_crew)
data['latest_age'] = data['age'].apply(get_latest_age)
data['latest_age']= data['latest_age'].fillna(value=np.nan)
# data.head(10)

In [21]:
df_age_bounty = data.dropna(subset=['latest_age', 'last_bounty'])
df_age_bounty['latest_age'] = df_age_bounty['latest_age'].astype('int')

fig = px.scatter(x = df_age_bounty['latest_age'], y=df_age_bounty['last_bounty'], color = df_age_bounty['Name'])
fig.update_xaxes(tickangle=0)
fig.show()

<ipython-input-21-2d76fe4d423c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
fig = px.scatter(x = df_age_bounty['latest_age'], y=df_age_bounty['last_bounty'], color = df_age_bounty['main_crew'])
fig.update_xaxes(tickangle=0)
fig.show()